# Load data related to power electronics

In [ ]:
from datasets import load_dataset

print("[INFO] Loading dataset from HuggingFace: IDEALLab/power_electronics_v0")
ds = load_dataset("IDEALLab/power_electronics_v0")
{"train": ds["train"].to_pandas(), "val": ds["val"].to_pandas(), "test": ds["test"].to_pandas()}
df_test = ds["test"].to_pandas()

# Data Analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Compute log(r + 1e-8)
log_r = np.log(df_test[" Voltage_Ripple"])

# Histogram
plt.figure(figsize=(7, 5))
plt.hist(log_r, bins=200, alpha=0.7)
plt.xlabel("log(r)")
plt.ylabel("Count")
plt.title("Histogram of log(r)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Compute log(r + 1e-8)
log_g = np.log(df_test[" DcGain"])

# Histogram
plt.figure(figsize=(7, 5))
plt.hist(log_g, bins=200, alpha=0.7)
plt.xlabel("log(g)")
plt.ylabel("Count")
plt.title("Histogram of log(g)")
plt.show()

In [ ]:
df_test.columns

In [ ]:
df_test.describe()

# Hyperparameter tuning with ax platform and botorch in backend

## Output: g

In [ ]:
!python ./engiopt/surrogate_model/bayes_optimize.py \
    --problem_id "power_electronics" \
    --target_col "DcGain" \
    --log_target \
    --params_cols '["initial_design_0","initial_design_1","initial_design_2","initial_design_3","initial_design_4","initial_design_5","initial_design_6","initial_design_7","initial_design_8","initial_design_9"]' \
    --strip_column_spaces \
    --flatten_columns '["initial_design"]' \
    --n_epochs 5 \
    --patience 3 \
    --seed 18 \
    --track \
    --wandb_project "engiopt_sm_hypertun_DcGain" \
    --wandb_entity smassoudi-eth-z-rich \
    --n_ensembles 1 \
    --save_model \
    --model_output_dir "my_models" \
    --test_model \
    --device "mps" \
    --total_trials 5 \
    --learning_rate_bounds 1e-5 1e-3 \
    --hidden_layers_choices 2 3 4 5 \
    --hidden_size_choices 16 32 64 128 256 \
    --batch_size_choices 8 16 32 64 128 \
    --l2_lambda_bounds 1e-6 1e-3 \
    --activation_choices "relu" "tanh"

# Hyperparameter tuning with ax platform and botorch in backend

## Output: r

In [ ]:
!python ./engiopt/surrogate_model/bayes_optimize.py \
    --problem_id "power_electronics" \
    --target_col "Voltage_Ripple" \
    --log_target \
    --params_cols '["initial_design_0","initial_design_1","initial_design_2","initial_design_3","initial_design_4","initial_design_5","initial_design_6","initial_design_7","initial_design_8","initial_design_9"]' \
    --strip_column_spaces \
    --flatten_columns '["initial_design"]' \
    --n_epochs 10 \
    --patience 5 \
    --seed 18 \
    --track \
    --wandb_project "engiopt_sm_hypertun_Voltage_Ripple" \
    --wandb_entity smassoudi-eth-z-rich \
    --n_ensembles 1 \
    --save_model \
    --model_output_dir "my_models" \
    --test_model \
    --device "mps" \
    --total_trials 3 \
    --learning_rate_bounds 1e-5 1e-3 \
    --hidden_layers_choices 2 3 4 5 \
    --hidden_size_choices 16 32 64 128 256 \
    --batch_size_choices 8 16 32 64 128 \
    --l2_lambda_bounds 1e-6 1e-3 \
    --activation_choices "relu" "tanh"

# Running a test

In [ ]:
!python ./engiopt/surrogate_model/mlp_tabular_only.py \
    --problem_id "power_electronics" \
    --target_col "DcGain" \
    --log_target \
    --params_cols '["initial_design_0","initial_design_1","initial_design_2","initial_design_3","initial_design_4","initial_design_5","initial_design_6","initial_design_7","initial_design_8","initial_design_9"]' \
    --l2_lambda 1e-6 \
    --strip_column_spaces \
    --flatten_columns '["initial_design"]' \
    --learning_rate 4e-4 \
    --lr_decay 0.95 \
    --activation "relu" \
    --hidden_layers 5 \
    --hidden_size 128 \
    --n_epochs 30 \
    --batch_size 16 \
    --patience 20 \
    --scale_target \
    --track \
    --wandb_project "engiopt_sm_test" \
    --wandb_entity smassoudi-eth-z-rich \
    --seed 18 \
    --n_ensembles 2 \
    --save_model \
    --model_output_dir "my_models" \
    --test_model \
    --device "mps"

# Ensemble Creation to predict g

In [ ]:
!python ./engiopt/surrogate_model/mlp_tabular_only.py \
    --problem_id "power_electronics" \
    --target_col "DcGain" \
    --log_target \
    --params_cols '["initial_design_0","initial_design_1","initial_design_2","initial_design_3","initial_design_4","initial_design_5","initial_design_6","initial_design_7","initial_design_8","initial_design_9"]' \
    --l2_lambda 1e-3 \
    --strip_column_spaces \
    --flatten_columns '["initial_design"]' \
    --learning_rate 1e-3 \
    --lr_decay 0.95 \
    --activation "relu" \
    --hidden_layers 3 \
    --hidden_size 256 \
    --n_epochs 150 \
    --batch_size 32 \
    --patience 50 \
    --scale_target \
    --track \
    --wandb_project "engiopt_sm_ensemble_DcGain" \
    --wandb_entity smassoudi-eth-z-rich \
    --seed 18 \
    --n_ensembles 7 \
    --save_model \
    --model_output_dir "my_models" \
    --test_model \
    --device "mps"

# Ensemble Creation to predict r

In [ ]:
!python ./engiopt/surrogate_model/mlp_tabular_only.py \
    --problem_id "power_electronics" \
    --target_col "Voltage_Ripple" \
    --log_target \
    --params_cols '["initial_design_0","initial_design_1","initial_design_2","initial_design_3","initial_design_4","initial_design_5","initial_design_6","initial_design_7","initial_design_8","initial_design_9"]' \
    --l2_lambda 1e-3 \
    --strip_column_spaces \
    --flatten_columns '["initial_design"]' \
    --learning_rate 0.00032 \
    --lr_decay 0.95 \
    --activation "relu" \
    --hidden_layers 4 \
    --hidden_size 256 \
    --n_epochs 150 \
    --batch_size 8 \
    --patience 50 \
    --scale_target \
    --track \
    --wandb_project "engiopt_sm_ensemble_Voltage_Ripple" \
    --wandb_entity smassoudi-eth-z-rich \
    --seed 18 \
    --n_ensembles 7 \
    --save_model \
    --model_output_dir "my_models" \
    --test_model \
    --device "mps"

# Running inference on one model

In [ ]:
import sys

import numpy as np
import pandas as pd
import torch

import engiopt.surrogate_model.model_pipeline
from engiopt.surrogate_model.model_pipeline import ModelPipeline

sys.modules["model_pipeline"] = engiopt.surrogate_model.model_pipeline


if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# 1) Load pipeline
pipeline = ModelPipeline.load("my_models/final_pipeline_engiopt__mlp_tabular__18__1744568179_Voltage_Ripple.pkl")

# 2) Prepare new raw data in a DataFrame
raw_data = df_test

# 3) Predict
# Since the pipeline now expects raw input, simply pass the DataFrame.
y_pred = pipeline.predict(raw_data, batch_size=64, device=device)
print("Predictions:", y_pred)

# 4) Evaluate (if you have ground-truth values)
# The evaluate method also expects raw data.
y_true = raw_data[" Voltage_Ripple"]  # For example, substitute with your actual ground-truth array
print("Truth:", y_true)
metrics = pipeline.evaluate(raw_data, y_true, batch_size=64, device=device, metrics=["mse", "rmse", "rel_err", "mae"])
print("Evaluation metrics:", metrics)

# Pymoo optimization

### Run the Optimization

In [ ]:
!python ./engiopt/surrogate_model/run_pe_optimization.py \
  --model_gain_path my_models/final_pipeline_engiopt__mlp_tabular__18__1744565887_DcGain.pkl \
  --model_ripple_path my_models/final_pipeline_engiopt__mlp_tabular__18__1744568179_Voltage_Ripple.pkl \
  --device "mps" \
  --pop_size 500 \
  --n_gen 100 \
  --seed 18 \
  --track \
  --wandb_entity smassoudi-eth-z-rich \
  --wandb_project engibench_sm_mooga

### Visualize the Pareto front

In [ ]:
import plotly.express as px

# Load the combined Pareto front CSV (or use your existing DataFrame)
df_front = pd.read_csv("results/pareto_front.csv")

# Create a scatter plot:
#   - x-axis: predicted r (objective f_r)
#   - y-axis: absolute deviation |g-0.25| (objective f_abs_g_minus_0.25)
#   - hover_data: shows the design variables (C1...T1)
fig = px.scatter(
    df_front,
    x="f_r",
    y="f_abs_g_minus_0.25",
    hover_data=["C1", "C2", "C3", "C4", "C5", "C6", "L1", "L2", "L3", "T1"],
    title="Pareto Front of Power Electronics Design Optimization",
    labels={"f_r": "Predicted r", "f_abs_g_minus_0.25": "|g - 0.25|"},
)

# Optional: improve layout and add interactivity
fig.update_traces(marker={"size": 10, "color": "red", "opacity": 0.8})
fig.update_layout(hovermode="closest")

fig.show()